# Importation des packages

In [1]:
import numpy as np
import pandas as pd
import cv2
from imutils import contours
import os
import matplotlib.pyplot as plt
from glob import glob

# Variables d'environnements

In [2]:
imprime_ecran_path = 'imprime_ecran'
matrix_convert_path = 'matrix_convert'
train_data = 'training_data'

# Fonctions

In [3]:
def extract_roi(tetris_game:np.ndarray, image_path:str, folder_name:str)->tuple:
    """ Extract the matrix from a tetris image and the next tetromino"""

    matrix_path = f'{matrix_convert_path}/{folder_name}/{image_path.split("/")[-1][:-4]}'
    
    roi_matrix = None
    
    original = tetris_game.copy()
    gray = cv2.cvtColor(original,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    (cnts, 
     _) = contours.sort_contours(cnts, method="left-to-right")
    num = 0
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        if w > 5:
            roi = original[y+9:y+h-9, x+9:x+w-9]

            if num == 0:
                roi_matrix = roi

                cv2.imwrite(f'{matrix_path}_roi_{num}.png', roi_matrix)

            num += 1
            
    return roi_matrix

In [4]:
def roi_to_training_example(roi_image:np.ndarray)->np.ndarray:
    

    gray = cv2.cvtColor(roi_image,cv2.COLOR_BGR2GRAY)
    thresh_gray = cv2.threshold(gray, 0, 255, cv2.THRESH_TOZERO)[1]
    resize_image = cv2.resize(thresh_gray, (10, 20), interpolation = cv2.INTER_AREA)

    thresh_resize_image = np.where(resize_image>50, 255, 0)

    return thresh_resize_image


In [5]:
def find_tetromino_column(matrix_x:np.ndarray, matrix_y:np.ndarray)->int:
    
    matrix_with_last_tetromino = matrix_y[2:, :]-matrix_x[2:, :] 
    
    column_with_tetromino = (np.sum(matrix_with_last_tetromino, axis=0)!=0)
    
    first_column_with_tetromino = column_with_tetromino.argmax(axis=0)
    
    return first_column_with_tetromino

In [6]:
def find_tetromino(matrix:np.ndarray)->np.ndarray:

    column_with_tetromino = (np.sum(matrix, axis=0)!=0)
    
    first_x_with_tetromino = column_with_tetromino.argmax(axis=0)    
    
    last_x_with_tetromino = np.max(np.nonzero(column_with_tetromino))
    

    line_with_tetromino = (np.sum(matrix, axis=1)!=0)  
    
    first_y_with_tetromino = line_with_tetromino.argmax(axis=0)  
    
    last_y_with_tetromino = np.max(np.nonzero(line_with_tetromino))

    return matrix[first_y_with_tetromino:last_y_with_tetromino+1, first_x_with_tetromino:last_x_with_tetromino+1]

In [7]:
def find_tetromino_rotation(matrix_x:np.ndarray, matrix_y:np.ndarray)->int:
    
    input_tetromino = find_tetromino(matrix_x[:2, :])
    
    matrix_with_last_tetromino = matrix_y[2:, :]-matrix_x[2:, :]
    
    output_tetromino = find_tetromino(matrix_with_last_tetromino)
    
    if input_tetromino.shape == output_tetromino.shape and np.all(np.equal(input_tetromino, input_tetromino)):
        return 0
    elif np.rot90(input_tetromino, k=1).shape == output_tetromino.shape and np.all(np.equal(input_tetromino, input_tetromino)):
        return 1
    elif np.rot90(input_tetromino, k=2).shape == output_tetromino.shape and np.all(np.equal(input_tetromino, input_tetromino)):
        return 2
    elif np.rot90(input_tetromino, k=3).shape == output_tetromino.shape and np.all(np.equal(input_tetromino, input_tetromino)):
        return 3

# Main

In [15]:
data_frame = pd.DataFrame([], columns=('name', 'training_path', 'column', 'rotation'))

for path_image in glob(f'{imprime_ecran_path}/X/*'):

        path_x_image = path_image.replace("\\", "/")
        path_y_image = f"{imprime_ecran_path}/Y/"+path_image.replace("\\", "/").split('/')[-1]

        x_image = cv2.imread(path_x_image)
        y_image = cv2.imread(path_y_image)

        roi_x_matrix = extract_roi(x_image, path_x_image, 'X')
        roi_y_matrix = extract_roi(y_image, path_y_image, 'Y')       
        
        matrix_training_x_example = roi_to_training_example(roi_x_matrix)
        matrix_training_y_example = roi_to_training_example(roi_y_matrix)
                
        y_column = find_tetromino_column(matrix_training_x_example, matrix_training_y_example)
        
        y_rotation = find_tetromino_rotation(matrix_training_x_example, matrix_training_y_example)
        
        path_training_data = f'{train_data}/X/'+path_x_image.split('/')[-1]
                                               
        cv2.imwrite(path_training_data, matrix_training_x_example)
        
        new_row = {"name":path_training_data.split('_')[-1][:-4],
                   "training_path":path_training_data,
                   "column":y_column,
                   "rotation":y_rotation}
        
        data_frame = data_frame.append(new_row, ignore_index=True)
        
data_frame.reset_index(drop=True)
data_frame.to_csv(f'{train_data}/data.csv')

imprime_ecran/X/0_hat.png
imprime_ecran/Y/0_hat.png
imprime_ecran/X/10_right-snake.png
imprime_ecran/Y/10_right-snake.png
imprime_ecran/X/11_left-snake.png
imprime_ecran/Y/11_left-snake.png
imprime_ecran/X/12_square.png
imprime_ecran/Y/12_square.png
imprime_ecran/X/13_right-snake.png
imprime_ecran/Y/13_right-snake.png
imprime_ecran/X/14_right-gun.png
imprime_ecran/Y/14_right-gun.png
imprime_ecran/X/15_long.png
imprime_ecran/Y/15_long.png
imprime_ecran/X/16_left-gun.png
imprime_ecran/Y/16_left-gun.png
imprime_ecran/X/17_right-gun.png
imprime_ecran/Y/17_right-gun.png
imprime_ecran/X/18_right-snake.png
imprime_ecran/Y/18_right-snake.png
imprime_ecran/X/19_long.png
imprime_ecran/Y/19_long.png
imprime_ecran/X/1_hat.png
imprime_ecran/Y/1_hat.png
imprime_ecran/X/20_left-gun.png
imprime_ecran/Y/20_left-gun.png
imprime_ecran/X/21_square.png
imprime_ecran/Y/21_square.png
imprime_ecran/X/22_right-gun.png
imprime_ecran/Y/22_right-gun.png
imprime_ecran/X/23_left-snake.png
imprime_ecran/Y/23_left-sn

In [ ]:
cv2.imread('imprime_ecran/Y/10_right-snake.png')

In [ ]:
from PIL import ImageGrab
import numpy as np
import cv2
while(True):
    img = ImageGrab.grab(bbox=(370,120,1050,780)) #bbox specifies specific region (bbox= x,y,width,height)
    img_np = np.array(img)
    frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    cv2.imshow("test", frame)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
from PIL import ImageGrab


screen = ImageGrab.grab(bbox=(651,190,1250,830))


screen.save('test.png', 'PNG') 